<h1><center> Python project proposal </center></h1>
<h2><center>Current car prices from bazos.cz</center></h2>
<h3><center>Daniel Brosko, Vojtěch Suchánek</center></h3>

Our goal is to web-scrape advertisements listed on website bazos.cz, which is currently one of the most used websites for selling used cars in Czech republic. It has more than 15 000 car adds daily. On the other hand, it has really poor search options, which pretty much complicates searching for desired car based on your parameters.

We are going to code algorithm, which will scan adds for the current day, pick those, which fulfill our conditions on date and car type and save their links. Then we will go to each link and save the text of the add. Then we will try to analyze the text of the add to find our parameters. In the end, we will perform some visualizations of distribution of price and selected parameters and try to select some underprices offers.

This task might be better to perform continuouslly, for example each hour, to not overload the website. This approach might also allow for longer time period to analyze the data. But since this project should be designed as one-time run, we decided to limit the data to only current date.

In [50]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
bots = requests.get('https://auto.bazos.cz/robots.txt')
print(bots.text)

User-agent: *
Disallow: /*humkreis
Disallow: /*order=
Disallow: /*cenaod=
Disallow: /*vkm=
Disallow: /*gal=
Disallow: /suggest.php
Disallow: /suggestpsc.php
Disallow: /suggestpscinsert.php
Disallow: /overeni.php
Disallow: /detailtel.php
Disallow: /agent.php
Disallow: /favourite.php
Disallow: /kmailn.php
Disallow: /upload.php
Disallow: /zip.php
Disallow: /smazat/

User-agent: Mediapartners-Google
Disallow:

User-agent: trovitBot
Disallow: /

User-agent: AhrefsBot
Disallow: /

User-agent: pricebot
Disallow: /

User-agent: magpie-crawler
Disallow: /

User-agent: SemrushBot
Disallow: /

User-Agent: trendictionbot
Disallow: /

User-agent: SemrushBot
Disallow: /

User-Agent: trendictionbot
Disallow: /

User-agent: BLEXBot
Disallow: /

User-agent: Riddler
Disallow: /

Sitemap: https://auto.bazos.cz/sitemap.php


From the robots page we can see that our actions done in our projects are allowed, since we are not gonna use these search commands.

Here we will get the main page from car section of bazos.

In [26]:
number_sequence = range(20, 400, 20) #start, stop (not included), step
main_url_list = list()
main_url_list.append('https://auto.bazos.cz/')

for i in range(0,len(number_sequence)):
    url = f'https://auto.bazos.cz/{number_sequence[i]}/'
    main_url_list.append(url)
    
print(main_url_list)

['https://auto.bazos.cz/', 'https://auto.bazos.cz/20/', 'https://auto.bazos.cz/40/', 'https://auto.bazos.cz/60/', 'https://auto.bazos.cz/80/', 'https://auto.bazos.cz/100/', 'https://auto.bazos.cz/120/', 'https://auto.bazos.cz/140/', 'https://auto.bazos.cz/160/', 'https://auto.bazos.cz/180/', 'https://auto.bazos.cz/200/', 'https://auto.bazos.cz/220/', 'https://auto.bazos.cz/240/', 'https://auto.bazos.cz/260/', 'https://auto.bazos.cz/280/', 'https://auto.bazos.cz/300/', 'https://auto.bazos.cz/320/', 'https://auto.bazos.cz/340/', 'https://auto.bazos.cz/360/', 'https://auto.bazos.cz/380/']


In [30]:
soup_list = list()
for url in main_url_list:
    page = requests.get(url)
    soup_list.append(BeautifulSoup(page.text, 'html'))


In [31]:
print(soup_list)

[<!DOCTYPE html>
<html lang="cs">
<head>
<title>Auto bazar | Bazoš.cz</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Auto bazar zdarma. Vybírejte z 343 999 inzerátů. Prodej snadno a rychle na Bazoši. Přes půl milionů uživatelů za den. Najděte co potřebujete." name="description"/><link href="https://auto.bazos.cz/" rel="canonical"/><meta content="1055875657" property="fb:admins"/>
<link href="https://www.bazos.cz/bazos60s.css" rel="stylesheet" type="text/css"/><link as="image" href="https://www.bazos.cz/obrazky/bazos.svg" rel="preload"/>
<link href="https://www.bazos.cz/bazosprint.css" media="print" rel="stylesheet" type="text/css"/>
<link href="https://www.bazos.cz/favicon.ico" rel="shortcut icon"/>
<link href="https://www.bazos.cz/obrazky/icon-192x192.png" rel="icon" sizes="192x192"/>
<link href="https://www.bazos.cz/apple-touch-icon.png" rel="apple-touch-icon"/>
<link href="https://www.bazos.cz/rss.php?rub=au" rel="alternate" title="Auto R

There are 20 adds plus other page elements. Lets pick just one add to see their construction.

There are several elements, which are important to us. In this section, we are interested in add id, which we are able to extract from href in element a on the second line. Its the number after "inzerat/". We are also interested in the href itself, since it is our link to follow to the actual add. Another element important to us is h2 class = "nadpis", which contains title of the add, from which we can extract type of car. And the last interesting element is span with class "velikost10", which reffers to date uploaded.

Now we can follow the link to the actual add page to see its content.

In [52]:
list_of_offers_url = list()
for element in soup_list:
    x = element.find('div', {'class':'inzeraty inzeratyflex'})
    y = x.find('div', {'class':'inzeratynadpis'})
    z = y.find('a')
    w = z.get('href')
    list_of_offers_url.append(f'https://auto.bazos.cz{w}')

print(list_of_offers_url)

['https://auto.bazos.cz/inzerat/152272573/vw-passat-b8-20tdi-110kw-aid12-tazne-keyless-full-led-navi.php', 'https://auto.bazos.cz/inzerat/152243437/1987-nissan-300zx-30-v6-z31-manual.php', 'https://auto.bazos.cz/inzerat/152210839/audi-a4-avant-20-tdi-110-kw-6-rychl-navi-xenonyalu.php', 'https://auto.bazos.cz/inzerat/152151328/octavia.php', 'https://auto.bazos.cz/inzerat/152094047/seat-ateca-16tdi-85kw-kessy-led-kamera-navi-nakup-v-klidu.php', 'https://auto.bazos.cz/inzerat/152059846/audi-q5-20-tdi-125kw-quattro-panorama-s-line.php', 'https://auto.bazos.cz/inzerat/152036240/vw-passat-b8-20tdi-110kw-dsg-tazne-kamera-nakup-v-klidu.php', 'https://auto.bazos.cz/inzerat/151986406/skoda-superb-20-tdi-dsg-sportline-cz-pano-dph-zaruka.php', 'https://auto.bazos.cz/inzerat/151939202/skoda-octavia-3-20tdi-110kw-dsg-acc-navi-nakup-v-klidu.php', 'https://auto.bazos.cz/inzerat/151908773/vw-touran-14tsi-110kw-kamera-tempomat-tazne-nakup-v-klidu.php', 'https://auto.bazos.cz/inzerat/151830914/superb-iii

In [ ]:
#similar as above for class = 'velikos10', 'nadpis', ...
# also the code below will be automatized using for loops, OOP, and etc.

In [31]:
#continue here
add_page = requests.get('https://auto.bazos.cz/inzerat/151109412/volkswagen-golf-gtd-135kw-manual-vyhrev-navi-led.php')

In [33]:
soup_add = BeautifulSoup(add_page.text, 'html')

In [34]:
print(soup_add.find('div', {'class':'popisdetail'}))

<div class="popisdetail">Volkswagen Golf - zachovalý vůz<br/>
GTD, 6-ti stupňová manuální převodovka, vyhřívaná sedadla, satelitní navigace, LED denní svícení<br/>
<br/>
Do provozu 10/2013<br/>
Najeto: 161 936 km<br/>
Objem: 1968 ccm<br/>
Výkon: 135kW (183PS)<br/>
Druh paliva: nafta<br/>
Stav: perfektní<br/>
Karoserie: hatchback<br/>
Barva: červená<br/>
Počet míst: 5<br/>
Počet dveří: 5<br/>
<br/>
VIN: WVWZZZAUZEW112045<br/>
<br/>
Výbava:<br/>
<br/>
6 rychlostních stupňů; 6x airbag; ABS; adaptivní tempomat; alu kola; aut. klimatizace; autorádio; AUX; bi-xenonové světlomety; bluetooth; CD přehrávač; centrál dálkový; centrální zamykání; deaktivace airbagu spolujezdce; dělená zadní sedadla; denní svícení; dvouzónová klimatizace; el. okna; el. sklopná zrcátka; el. zrcátka; elektronická ruční brzda; hands free; imobilizér; isofix; LED denní svícení; manuální převodovka; mlhovky; multifunkční volant; nastavitelný volant; ostřikovače světlometů; palubní počítač; podélný posuv sedadel; polohov

Here we can see the description, which we will use to extract parameters of the car. This will be one of the most difficult parts of the project to extract the relevant text in unformated text. There is no official format to the text, so we will have to find a way how to extract this information from various formats.

After retrieving these informations, we will use analysis of the data got from this process.